In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Pinecone
import pinecone
from langchain_community.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_community.llms import CTransformers

In [2]:
PINECONE_API_KEY = "dcd526af-6f63-493b-8c71-7b2d43d75473"
PINECONE_API_ENV = "gcp-starter"

In [5]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data, glob= "*.pdf",
                    loader_cls=PyPDFLoader)
    
    dpcuments= loader.load()

    return documents